<a href="https://colab.research.google.com/github/bobsi1/Interviews/blob/main/%D0%94%D0%BB%D1%8F_%D1%81%D0%BE%D0%B1%D0%B5%D1%81%D0%B5%D0%B4%D0%BE%D0%B2%D0%B0%D0%BD%D0%B8%D0%B9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install langchain langchain-mistralai mistralai faiss-cpu==1.13.2

INFO: pip is looking at multiple versions of opentelemetry-sdk to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 59.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.3/487.3 kB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.3/160.3 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.0/208.0 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.9/65.9 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.0 MB/s eta 0:00:00
  Attempting uninstall: opentelemetry-proto
    Found existing installation: opentelemetry-proto 1.37.0
    Uninstalling opentelemetry-proto-1.37.0:
      Successfully uninstalled opentelemetry-proto-1.37.0
  Attempting uninstall: opentelemetry-exporter-otlp-proto-common
    Found existi

In [4]:
from google.colab import userdata
from mistralai.client import MistralClient
from mistralai import Mistral, UserMessage, SystemMessage
import json
import re
import faiss
from sentence_transformers import SentenceTransformer

MISTRAL_API_KEY = userdata.get("MISTRAL_API_KEY")
client = Mistral(api_key=MISTRAL_API_KEY)

def mistral_chat(system, user, history, mentors_thoughts = None):
  if mentors_thoughts is None:
    mentors_thoughts = []
  messages_h = [{"role" : "system", "content" : system}] + history +  mentors_thoughts + [{"role" : "user",  "content" : user}]
  resp = client.chat.complete(
    model="mistral-large-latest",
    temperature = 0.4,
    messages = messages_h

  )
  return resp.choices[0].message.content.strip()

In [5]:
context_input = input("""Привет! Поделись, пожалуйста, информацией о себе в таком формате:
Имя:
Позиция:
Грейд:
Опыт:
""")


context = {
    "name" : context_input.split('Имя: ')[1].split('Позиция: ')[0].strip(),
    "pos" : context_input.split('Позиция: ')[1].split('Грейд: ')[0].strip(),
    "grade" : context_input.split('Грейд: ')[1].split('Опыт: ')[0].strip(),
    "exp" : context_input.split('Опыт: ')[1].strip(),
    "history" : []
}

state = {
    "participant_name" : "Рубцова Вера Вадимовна",
    "turns" : [],
    #"turn_id" : 0,
    "final_feedback": None
}

def add_state(user_message, internal_thoughts, agent_visible_message):
  turn_id = len(state["turns"]) + 1
  state["turns"].append({
      "turn_id" : turn_id,
      "user_message" : user_message,
      "internal_thoughts": internal_thoughts,
      "agent_visible_message" : agent_visible_message})

def get_history(n = 20):
  return context['history'][-n:]


Привет! Поделись, пожалуйста, информацией о себе в таком формате: 
Имя: 
Позиция: 
Грейд: 
Опыт:
Имя: Алекс Позиция: Backend Developer Грейд: Junior  Опыт: Пет-проекты на Django, немного SQL.


In [6]:
interviwer_system = f"""Ты проводишь техническое собьеседование для кандидата на вакансию.
  Веди себя профессионально и дружелюбно.
  Задавай один вопрос за раз.
  Если кандидат начинает задавать встречные вопросы уходя от темы, ответь сухо и кратко и сразу вернись к техническим вопросам.  Учитывай грейд опыт и направление кандидата, обращайся к нему по имени: Имя: {context["name"]}, Грейд: {context["grade"]}, Позиция: {context["pos"]}, Опыт: {context["exp"]}
  Не анализируй и не оценивай ответы кандидата, это задача ментора, твоя задача - задавать вопросы, учитывая рекомендации ментора"""
interviwer_system_que = f"""Ты проводишь техническое собьеседование для кандидата на вакансию.
  Верни одну короткую мысль. Если ты получил от ментора несколько вариантов вопросов кандидату - обьясни свой выбор. Если получил один - вырази свою точку зрения на анализ ментора"""

mentor_final_result = f"""Ты выступаешь ментором во время собеседования. Сформируй отчет на основе ответов и поведения кандидата по образцу:
А. Вердикт (Decision)
  ●	Grade: Уровень кандидата (Junior / Middle / Senior) на основе ответов.
  ●	Hiring Recommendation: (Hire / No Hire / Strong Hire).
  ●	Confidence Score: Насколько уверен в оценке (0-100%).
Б. Анализ Hard Skills (Technical Review)
Таблица или список тем, затронутых в интервью. - просто краткое название
  ●	Confirmed Skills: Темы, где кандидат дал точные ответы. Просто перечислить темы
  ●	Knowledge Gaps: Темы, где были допущены ошибки или кандидат сказал «не знаю». Тема - правильный ответ.
○	Важно: Здесь нужно привести правильный ответ только на те вопросы, которые пользователь завалил.
В. Анализ Soft Skills & Communication
  ●	Clarity: Насколько понятно излагает мысли.
  ●	Honesty: Пытался ли кандидат выкрутиться/соврать или честно признал незнание.
  ●	Engagement: Задавал ли встречные вопросы (если это было в сценарии).
Г. Персональный Roadmap (Next Steps)
  ●	Список конкретных тем/технологий, которые нужно подтянуть (на основе выявленных пробелов).
Ответ нужен просто текстом"""

mentor_system = f"""Ты выступаешь ментором во время собеседования.
  Ты должен оценивать ответы и поведение кандидата и формировать записи для интервьюра
  По шаблону:
  "краткое название темы вопроса интервьюера (под темой подразумевай раздел знаний)",
  "result" : "confirmed/gap",
  "right_answer" : "если result == gap, то приведи краткий верный ответ, иначе пусто",
  "opinion" : "расскажи кратко, что было хорошо и плохо. предложи следующий вопрос и обьясни почему его предложил"
  ВСЕГДА указывай следующий вопрос В КОНЦЕ своего текста так: Следующий вопрос: ...
  Учитывай грейд опыт и направление кандидата: Имя: {context["name"]}, Грейд: {context["grade"]}, Позиция: {context["pos"]}, Опыт: {context["exp"]}"""

In [8]:
from mistralai.models.no_response_error import NoResponseError
def interviwer(system, user, mentors_thoughts = None):
  int_ans = mistral_chat(system, user, get_history(n = 20), mentors_thoughts = mentors_thoughts)
  return int_ans

def mentor(system, user):
  men_ans = mistral_chat(system, user, get_history(n = 20))
  return men_ans

def main_function(user_message, last_que):
  que_ans = f"Вопрос: {last_que}\n Ответ {user_message}"
  internal_thoughts = mentor(mentor_system, que_ans)

  mentors_thoughts = [{"role" : "system",
                      "content" : "внутренние мысли ментора НЕЛЬЗЯ повторять или показывать кандидату" + internal_thoughts}]



  agent_visible_message = interviwer(interviwer_system, user_message, mentors_thoughts = mentors_thoughts)
  interviwer_que = interviwer(interviwer_system_que, f"Ты задал вопрос {agent_visible_message} Поясни на основе mentors_thoughts", mentors_thoughts = mentors_thoughts)
  internal_thoughts = f"[Observer]: {internal_thoughts} \n [Interviewer]: {interviwer_que}"
  add_state(user_message, internal_thoughts, last_que)

  context['history'].append({"role" : "user",
                            "content" : user_message})
  context['history'].append({"role" : "assistant",
                            "content" : agent_visible_message})

  return agent_visible_message

def start_interview(context_input):
  mentor_start = mentor(mentor_system + "Это первая информация по кандидату. Кратко подскажи интервьюеру с чего начать", context_input)
  mentor_start_thoughts = [{"role" : "system",
                          "content" : "внутренние мысли ментора НЕЛЬЗЯ повторять или показывать кандидату" + mentor_start}]
  int_ans = interviwer(interviwer_system, "Начни и задай первый вопрос", mentors_thoughts = mentor_start_thoughts)

  #add_state(context_input, mentor_start, int_ans)
  context['history'].append({"role" : "assistant",
                            "content" : int_ans})
  return int_ans

print("Для остановки напиши Стоп интервью")
first_que = start_interview(context_input)
last_que = first_que
print(first_que)

while True:
  user_message = input()
  if user_message == 'Стоп интервью':
    last = ""
    add_state(user_message, "Заканчиваем. Формируем отчет", last)
    fin = mentor(mentor_final_result, "Сформируй отчет")
    state["final_feedback"] = fin
    print(fin)
    break
  ans = main_function(user_message, last_que)
  last_que = ans
  print(ans)
with open('interview_log.json', 'w', encoding='utf-8') as s:
  json.dump(state, s, indent=4, ensure_ascii=False)

Для остановки напиши Стоп интервью
Алекс, давай теперь поговорим про связи между моделями.

Представь, что у модели `Order` есть связь с моделью `Product` через `ManyToManyField`. Как бы ты написал запрос, чтобы получить все заказы пользователя с `id=5` и сразу подгрузить связанные продукты, избежав проблемы N+1 запросов?


KeyboardInterrupt: Interrupted by user